# // Getting Individual Player Data
___
The first step in getting this project underway is going to be getting **massive** amounts of NFL data from the web. I will be working in this notebook to "show my work" and for others to learn how to if they're curious. Ultimately, I'll also turn it into a regular `.py` Python script that you can run if you're so inclined.

For that, we're going to rely on the `requests` and `BeautifulSoup` libraries to glean information from:

- [FantasyPros](https://www.fantasypros.com/)
- [Pro Football Reference](https://www.pro-football-reference.com/)
- [FFToday](http://www.fftoday.com/stats/)
- [The Football Database](https://www.footballdb.com/fantasy-football/index.html)

**Editor's Note:** After researching several different options for quite some time and attempting to use many of them, I will be going with **FantasyPros** for reasons outlined below.

In [36]:
# Importing our necessary libraries
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import requests
import re

from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep
%matplotlib inline

## FantasyPros
I'm choosing to go with this FantasyPros for the following reasons:
- You don't have to register to access the data
- Separately, I am currently a user and like their site overall
- Has available .5 PPR scoring data (FanDuel does .5 PPR so DFS appeal, and it will be most applicable for most leagues)
- The Football Database seemed like it was missing quite a few players from each week
- FFToday scoring was inconsistent in many places for .5 PPR scoring

#### Following the FanDuel vein, since Daily Fantasy Sports (DFS) is where the biggest value is going to be coming from, I will be scraping data for the following positions and referring to them from here on out by the name in parentheses:
1. Quarterback (QB)
2. Running Back (RB)
3. Wide Receiver (WR)
4. Tight End (TE)
5. Defense / Special Teams (DST)


> - For a more detailed breakdown on positional scoring, please reference FanDuel's scoring and rules reference [here](https://www.fanduel.com/rules).
- For a quick reference on what each column means, check out ESPN's stat reference [here](http://www.espn.com/nfl/news/story?id=2128923).

___
### Scraping for our QBs
Going to walk through the specific steps for this first position then blast through the others.

• Let's make some empty lists to throw all of our data into:

In [37]:
player = []
pass_comp = []
pass_att = []
pass_pct = []
pass_yds = []
yds_per_att = []
pass_TD = []
pass_INT = []
sacks_taken = []
rush_att = []
rush_yds = []
rush_TD = []
fumbles_lost = []
active = []
fpoints = []
fpoints_g = []
own_pct = []

In [38]:
# Make sure to incoporate this into the scrape!
week = []
year = []

• And a list of those lists to make `dataframe` creation nice and easy:

In [39]:
qb_stats_lists = [player, pass_comp, pass_att, pass_pct, pass_yds, 
                  yds_per_att, pass_TD, pass_INT, sacks_taken, 
                  rush_att, rush_yds, rush_TD, fumbles_lost, 
                  active, fpoints, fpoints_g, own_pct]



• Doing the actual scrape by going through each week of the season (via each URL iteration), grabbing the right `table`, iterating through each `row` and then `cell`, and putting that data into the appropriate `lists`.

In [70]:
# First let's add a quick little variable for our current week in the 2018 NFL season
res = requests.get('https://www.fantasypros.com/nfl/')
soup = BeautifulSoup(res.content, 'lxml')
current_week = [int(number) for number in soup.find_all('span', {'class': 'item-title'})[8].text if number.isdigit()][0]

In [41]:
for week_number in range(1,current_week):

    res = requests.get('https://www.fantasypros.com/nfl/stats/qb.php?week={}&scoring=HALF&range=week'.format(week_number))
    soup = BeautifulSoup(res.content, 'lxml')

    for row in soup.find('div', {'class':'mobile-table'}).find('tbody').find_all('tr'):
        cells = row.find_all('td')
        for index, selection in enumerate(qb_stats_lists):
            selection.append(cells[index].text.lstrip().strip())
        # I want "week" to become a unique identifier to bring in opponents
        week.append(week_number)
    # And adding a 1 second delay 
    sleep(1)

• Let's make an empty `dataframe` and then fill it using our list of lists:

In [42]:
qbdf = pd.DataFrame(columns= ['player', 'pass_comp', 'pass_att', 'pass_pct',
                              'pass_yds', 'yds_per_att', 'pass_TD', 'pass_INT', 
                              'sacks_taken', 'rush_att', 'rush_yds', 'rush_TD', 
                              'fumbles_lost', 'active', 'fpoints', 'fpoints_g', 'own_pct'])
qbdf

,player,pass_comp,pass_att,pass_pct,pass_yds,yds_per_att,pass_TD,pass_INT,sacks_taken,rush_att,rush_yds,rush_TD,fumbles_lost,active,fpoints,fpoints_g,own_pct


In [43]:
for index, column in enumerate(qbdf.columns):
    qbdf[column] = qb_stats_lists[index]

• Let's add on our `week` and `year` information as well as a separate `team` column from our `player` column (to later be used to bring in opponent data).

In [44]:
# Week
qbdf['week'] = week
qbdf['week'] = qbdf.week.astype(str)

In [45]:
team = []
for individual in qbdf['player']:
    team.append(re.findall('\(([^\)]+)\)', individual)[0])
qbdf['team'] = team

• And let's just clean up that `player` name a bit too with another `regex`:

In [46]:
qbdf['player'] = [ind[0] for ind in [re.findall('^.*?(?=\s\()', quarterback) for quarterback in qbdf['player']]]

• While we're at the cleanup, we're going to need to do some with the `own_pct`:

In [47]:
qbdf['own_pct'] = qbdf.own_pct.apply(lambda x: x.strip('%'))
qbdf.own_pct.replace(to_replace='', value='0.0', inplace=True)

• Since most if not all of these values were actually read in as `strings`, we'll need to convert them to numeric columns (with the exceptions of `player`, `team`, and `week`.

In [48]:
for x in [col for col in qbdf.columns if col not in ['player','week', 'team', 'year']]:
    qbdf[x] = qbdf[x].astype(float)

**Since we're going to likely be wanting to clean up our other dataframes in the same way, let's make a function to clean up our future code:**

In [49]:
def table_cleaner(df):
    # Adding 'week' as a string
    df['week'] = week
    df['week'] = df.week.astype(str)
    
    # Cleaning 'team'
    team = []
    for individual in df['player']:
        team.append(re.findall('\(([^\)]+)\)', individual)[0])
    df['team'] = team
    
    # Removing team from 'player'
    df['player'] = [ind[0] for ind in [re.findall('^.*?(?=\s\()', person) for person in df['player']]]    
    
    # Converting 'own_pct' to float
    df['own_pct'] = df.own_pct.apply(lambda x: x.strip('%'))
    df.own_pct.replace(to_replace='', value='0.0', inplace=True)    
    
    # Converting everything else to float where appropriate
    for x in [col for col in df.columns if col not in ['player','week', 'team', 'year']]:
        df[x] = df[x].astype(float)
    
    return df

In [50]:
# Checking to see if we'll be able to manipulate it how we want
qbdf.sort_values('fpoints', ascending=False).head(10)

,player,pass_comp,pass_att,pass_pct,pass_yds,yds_per_att,pass_TD,pass_INT,sacks_taken,rush_att,rush_yds,rush_TD,fumbles_lost,active,fpoints,fpoints_g,own_pct,week,team
461,Mitch Trubisky,19.0,26.0,73.1,354.0,13.6,6.0,0.0,1.0,3.0,53.0,0.0,0.0,1.0,43.5,43.5,72.3,4,CHI
30,Ryan Fitzpatrick,21.0,28.0,75.0,417.0,14.9,4.0,0.0,0.0,12.0,36.0,1.0,0.0,1.0,42.3,42.3,7.0,1,TB
262,Drew Brees,39.0,49.0,79.6,396.0,8.1,3.0,0.0,1.0,3.0,7.0,2.0,0.0,1.0,40.5,40.5,98.5,3,NO
272,Matt Ryan,26.0,35.0,74.3,374.0,10.7,5.0,0.0,3.0,4.0,12.0,0.0,0.0,1.0,40.2,40.2,95.5,3,ATL
449,Jared Goff,26.0,33.0,78.8,465.0,14.1,5.0,0.0,1.0,2.0,7.0,0.0,0.0,1.0,39.3,39.3,93.7,4,LAR
125,Ben Roethlisberger,39.0,60.0,65.0,452.0,7.5,3.0,0.0,1.0,2.0,9.0,1.0,0.0,1.0,39.0,39.0,93.9,2,PIT
218,Patrick Mahomes,23.0,28.0,82.1,326.0,11.6,6.0,0.0,1.0,5.0,18.0,0.0,0.0,1.0,38.8,38.8,99.0,2,KC
418,Andrew Luck,40.0,62.0,64.5,464.0,7.5,4.0,0.0,4.0,3.0,11.0,0.0,1.0,1.0,35.7,35.7,93.2,4,IND
177,Kirk Cousins,35.0,48.0,72.9,425.0,8.9,4.0,1.0,2.0,2.0,5.0,0.0,0.0,1.0,33.5,33.5,96.6,2,MIN
834,Patrick Mahomes,28.0,39.0,71.8,358.0,9.2,4.0,1.0,2.0,4.0,45.0,0.0,0.0,1.0,32.8,32.8,99.0,7,KC


In [51]:
qbdf.week.unique()

array(['1', '2', '3', '4', '5', '6', '7', '8'], dtype=object)

Through Week 5, no player has posted 2 of the season's Top 10 QB performances — yay parity!

In [52]:
qbdf.groupby('player')['fpoints'].mean().sort_values(ascending=False)[0:10]

player
Patrick Mahomes    27.1125
Andrew Luck        21.4875
Matt Ryan          21.2750
Cam Newton         21.1000
Jared Goff         20.6500
Kirk Cousins       20.6250
Deshaun Watson     19.7625
Mitch Trubisky     19.7625
Drew Brees         18.6375
Tom Brady          18.6250
Name: fpoints, dtype: float64

Through Week 5, we see some familiar suspects in the Top 10 points per game, but also some young guns like Mahomes, Goff, Watson, Cousins, and a resurgence for Luck.

• Now let's dump everything we have so far into a `csv` file that we can save to our current working directory:

In [53]:
qbdf.to_csv('./data/qb_stats_2018.csv')

___
### Running Backs
(Note that the tables are structured significantly differently for each position otherwise we would be able to use one nice and clean single scrape).

In [54]:
player = []

rush_att = []
rush_yds = []
rush_ypc = []
long = []
over_20 = []
rush_TD = []

receptions = []
targets = []
rec_yds = []
rec_ypr = []
rec_TD = []

fumbles_lost = []
active = []
fpoints = []
fpoints_g = []
own_pct = []

week = []
year = []

In [55]:
rb_stats_lists = [player,rush_att,rush_yds,rush_ypc,long,over_20,rush_TD,
                  receptions,targets,rec_yds,rec_ypr,rec_TD,
                  fumbles_lost,active,fpoints,fpoints_g,own_pct]

for week_number in range(1,current_week):
    res = requests.get('https://www.fantasypros.com/nfl/stats/rb.php?week={}&scoring=HALF&range=week'.format(week_number))
    soup = BeautifulSoup(res.content, 'lxml')
    for row in soup.find('div', {'class':'mobile-table'}).find('tbody').find_all('tr'):
        cells = row.find_all('td')
        for index, selection in enumerate(rb_stats_lists):
            selection.append(cells[index].text.lstrip().strip())
        # I want "week" to become a unique identifier to bring in opponents
        week.append(week_number)
    sleep(1)

rbdf = pd.DataFrame(columns= ['player','rush_att','rush_yds','rush_ypc','long','over_20','rush_TD',
                  'receptions','targets','rec_yds','rec_ypr','rec_TD',
                  'fumbles_lost','active','fpoints','fpoints_g','own_pct'])
for index, column in enumerate(rbdf.columns):
    rbdf[column] = rb_stats_lists[index]

rbdf = table_cleaner(rbdf)

In [56]:
rbdf.to_csv('./data/rb_stats_2018.csv')

In [57]:
# Little spot check to make sure we're Gucci
rbdf.loc[rbdf['fpoints'] == rbdf.fpoints.max(), :]

,player,rush_att,rush_yds,rush_ypc,long,over_20,rush_TD,receptions,targets,rec_yds,rec_ypr,rec_TD,fumbles_lost,active,fpoints,fpoints_g,own_pct,week,team
188,Alvin Kamara,8.0,29.0,3.6,10.0,0.0,2.0,9.0,12.0,112.0,12.4,1.0,0.0,1.0,38.6,38.6,100.0,1,NO
994,Alvin Kamara,19.0,134.0,7.1,49.0,1.0,3.0,5.0,9.0,47.0,9.4,0.0,0.0,1.0,38.6,38.6,100.0,4,NO


___
### Wide Receivers

In [58]:
player = []

receptions = []
targets = []
rec_yds = []
rec_ypr = []
long = []
over_20 = []
rec_TD = []

rush_att = []
rush_yds = []
rush_TD = []

fumbles_lost = []
active = []
fpoints = []
fpoints_g = []
own_pct = []

week = []
year = []

In [59]:
wr_stats_lists = [player, receptions, targets, rec_yds, rec_ypr, long, over_20, rec_TD,
                  rush_att, rush_yds, rush_TD,
                  fumbles_lost, active, fpoints, fpoints_g, own_pct]

for week_number in range(1,current_week):
    res = requests.get('https://www.fantasypros.com/nfl/stats/wr.php?week={}&scoring=HALF&range=week'.format(week_number))
    soup = BeautifulSoup(res.content, 'lxml')
    for row in soup.find('div', {'class':'mobile-table'}).find('tbody').find_all('tr'):
        cells = row.find_all('td')
        for index, selection in enumerate(wr_stats_lists):
            selection.append(cells[index].text.lstrip().strip())
        # I want "week" to become a unique identifier to bring in opponents
        week.append(week_number)
    sleep(1)

wrdf = pd.DataFrame(columns= ['player', 'receptions', 'targets', 'rec_yds', 'rec_ypr', 'long', 'over_20', 'rec_TD',
                  'rush_att', 'rush_yds', 'rush_TD',
                  'fumbles_lost', 'active', 'fpoints', 'fpoints_g', 'own_pct'])
for index, column in enumerate(wrdf.columns):
    wrdf[column] = wr_stats_lists[index]

wrdf = table_cleaner(wrdf)

In [60]:
wrdf.to_csv('./data/wr_stats_2018.csv')

In [61]:
# Andddd our usual sanity check
wrdf.loc[wrdf['fpoints'] > 30, :]

,player,receptions,targets,rec_yds,rec_ypr,long,over_20,rec_TD,rush_att,rush_yds,rush_TD,fumbles_lost,active,fpoints,fpoints_g,own_pct,week,team
218,Tyreek Hill,7.0,8.0,169.0,24.1,58.0,5.0,2.0,2.0,4.0,0.0,0.0,1.0,32.8,32.8,100.0,1,KC
546,Stefon Diggs,9.0,13.0,128.0,14.2,75.0,1.0,2.0,1.0,1.0,0.0,0.0,1.0,31.4,31.4,99.8,2,MIN
1041,Calvin Ridley,7.0,8.0,146.0,20.9,75.0,1.0,3.0,1.0,9.0,0.0,0.0,1.0,37.0,37.0,83.6,3,ATL
1355,Cooper Kupp,9.0,11.0,162.0,18.0,70.0,2.0,2.0,0.0,0.0,0.0,0.0,1.0,32.7,32.7,93.3,4,LAR
1975,Davante Adams,10.0,16.0,132.0,13.2,38.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,30.2,30.2,99.9,6,GB
2055,Tyreek Hill,7.0,12.0,142.0,20.3,75.0,2.0,3.0,1.0,0.0,0.0,0.0,1.0,35.7,35.7,100.0,6,KC


___
### Tight Ends

In [62]:
player = []

receptions = []
targets = []
rec_yds = []
rec_ypr = []
long = []
over_20 = []
rec_TD = []

rush_att = []
rush_yds = []
rush_TD = []

fumbles_lost = []
active = []
fpoints = []
fpoints_g = []
own_pct = []

week = []
year = []

In [63]:
te_stats_lists = [player, receptions, targets, rec_yds, rec_ypr, long, over_20, rec_TD,
                  rush_att, rush_yds, rush_TD,
                  fumbles_lost, active, fpoints, fpoints_g, own_pct]

for week_number in range(1,current_week):
    res = requests.get('https://www.fantasypros.com/nfl/stats/te.php?week={}&scoring=HALF&range=week'.format(week_number))
    soup = BeautifulSoup(res.content, 'lxml')
    for row in soup.find('div', {'class':'mobile-table'}).find('tbody').find_all('tr'):
        cells = row.find_all('td')
        for index, selection in enumerate(te_stats_lists):
            selection.append(cells[index].text.lstrip().strip())
        # I want "week" to become a unique identifier to bring in opponents
        week.append(week_number)
    sleep(1)

tedf = pd.DataFrame(columns= ['player', 'receptions', 'targets', 'rec_yds', 'rec_ypr', 'long', 'over_20', 'rec_TD',
                  'rush_att', 'rush_yds', 'rush_TD',
                  'fumbles_lost', 'active', 'fpoints', 'fpoints_g', 'own_pct'])
for index, column in enumerate(tedf.columns):
    tedf[column] = te_stats_lists[index]

tedf = table_cleaner(tedf)

In [64]:
tedf.to_csv('./data/te_stats_2018.csv')

In [65]:
# Chiggity check it
tedf.loc[tedf['fpoints'] > 25, :]

,player,receptions,targets,rec_yds,rec_ypr,long,over_20,rec_TD,rush_att,rush_yds,rush_TD,fumbles_lost,active,fpoints,fpoints_g,own_pct,week,team
285,Travis Kelce,7.0,10.0,109.0,15.6,31.0,2.0,2.0,0.0,0.0,0.0,0.0,1.0,26.4,26.4,100.0,2,KC
668,Jared Cook,8.0,13.0,110.0,13.8,24.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,27.0,27.0,83.7,4,OAK
944,Eric Ebron,9.0,15.0,105.0,11.7,28.0,1.0,2.0,0.0,0.0,0.0,0.0,1.0,27.0,27.0,94.4,5,IND


___
### Defense / Special Teams

In [66]:
player = []

sacks = []
INTs = []
fum_rec = []
force_fum = []
def_td = []
safeties = []
spc_TD = []

active = []
fpoints = []
fpoints_g = []
own_pct = []

week = []
year = []

In [67]:
dst_stats_lists = [player, sacks, INTs, fum_rec, force_fum, 
                   def_td, safeties, spc_TD, 
                   active, fpoints, fpoints_g, own_pct]

for week_number in range(1,current_week):
    res = requests.get('https://www.fantasypros.com/nfl/stats/dst.php?week={}&scoring=HALF&range=week'.format(week_number))
    soup = BeautifulSoup(res.content, 'lxml')
    for row in soup.find('div', {'class':'mobile-table'}).find('tbody').find_all('tr'):
        cells = row.find_all('td')
        for index, selection in enumerate(dst_stats_lists):
            selection.append(cells[index].text.lstrip().strip())
        # I want "week" to become a unique identifier to bring in opponents
        week.append(week_number)
    sleep(1)

dstdf = pd.DataFrame(columns= ['player', 'sacks', 'INTs', 'fum_rec', 'force_fum', 
                   'def_td', 'safeties', 'spc_TD', 
                   'active', 'fpoints', 'fpoints_g', 'own_pct'])
for index, column in enumerate(dstdf.columns):
    dstdf[column] = dst_stats_lists[index]

dstdf = table_cleaner(dstdf)

In [68]:
dstdf.to_csv('./data/dst_stats_2018.csv')

In [69]:
dstdf.loc[dstdf.fpoints >= 20, :].sort_values('fpoints', ascending = False)

,player,sacks,INTs,fum_rec,force_fum,def_td,safeties,spc_TD,active,fpoints,fpoints_g,own_pct,week,team
201,Denver Broncos,6.0,3.0,2.0,3.0,2.0,0.0,0.0,1.0,32.0,32.0,0.0,7,DEN
107,Green Bay Packers,7.0,2.0,1.0,1.0,0.0,0.0,0.0,1.0,23.0,23.0,0.0,4,GB
134,Cincinnati Bengals,3.0,2.0,1.0,1.0,2.0,0.0,0.0,1.0,22.0,22.0,0.0,5,CIN
143,Kansas City Chiefs,5.0,4.0,1.0,1.0,1.0,0.0,0.0,1.0,22.0,22.0,0.0,5,KC
128,Arizona Cardinals,4.0,2.0,3.0,3.0,1.0,0.0,0.0,1.0,21.0,21.0,0.0,5,ARI
162,Baltimore Ravens,11.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,21.0,21.0,0.0,6,BAL
220,Los Angeles Rams,7.0,2.0,2.0,2.0,0.0,1.0,0.0,1.0,21.0,21.0,0.0,7,LAR
21,New York Jets,0.0,5.0,0.0,0.0,1.0,0.0,1.0,1.0,20.0,20.0,0.0,1,NYJ
242,New England Patriots,3.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,20.0,20.0,0.0,8,NE


___
# // Wrap-up
___
In this notebook, we brought in game-level stats for individual positions through the current week of the 2018 season and saved them to our local machine.

Overall, we need to bring in:
- ~~QBs~~
- ~~RBs~~
- ~~WRs~~
- ~~TEs~~
- ~~DSTs~~
- Opponents (including game location)
- Defensive stats
- Team play-calling
- Depth charts?
- Weather